### Creating new environement

In [ ]:
'''
conda update -n base conda &&
conda install -n base conda-libmamba-solver &&
conda config --set solver libmamba &&
source ~/anaconda3/bin/activate
'''

In [ ]:
'''
conda create -n nnunet_topcow2024 python==3.9.13 anaconda -y
conda activate nnunet_topcow2024
conda update -n nnunet_topcow2024 conda -y
'''


'''
conda create -n nnunet_topcow24 python==3.9.13 anaconda -y
conda activate nnunet_topcow24
conda update -n nnunet_topcow24 conda -y

### MODIFIED THIS FILE : /home/hasna/miniconda3/envs/nnunet_topcow24/lib/python3.9/site-packages/acvl_utils/cropping_and_padding/bounding_boxes.py
'''

In [ ]:
'''
# pip install -r requirements.txt
pip install -r /home/hasna/nnUNet_dir/requirements.txt
'''

In [ ]:
'''
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
pip install threadpoolctl==3.1.0
'''

In [ ]:
'''
export PYTHONPATH="${PYTHONPATH}:/home/hasna/miccai24_challenges/epvs_challenge/nnunet_dir"
'''

### *Preparing nnUNet*

#### *Installation*

In [ ]:
'''
TODO: Following instructions from:
https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/installation_instructions.md
pip install -e . 
'''

#### *Dataset format*

In [2]:
#libraries
import os
import sys
import json
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import nibabel as nib
import SimpleITK as sitk
import matplotlib.pyplot as plt

from glob import glob 
from tqdm import tqdm
from natsort import natsorted
from collections import OrderedDict
from scipy.ndimage import find_objects


##### Task 1 - CoW Segmentation

In [4]:
''' Binary MRA segmentation using MRA only '''

dataset_name = 'Dataset802_TopCoWSegBinMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.startswith('topcow_mr_'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data = 1*(data>0)
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [02:52<00:00,  1.45it/s]


In [2]:
''' Binary CTA segmentation using CTA only '''

dataset_name = 'Dataset809_TopCoWSegBinCTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.startswith('topcow_ct_'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data = 1*(data>0)
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [01:29<00:00,  2.78it/s]


In [6]:
''' Binary MRA segmentation using MRA only '''

dataset_name = 'Dataset810_TopCoWSegBinMRAnew'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.startswith('topcow_mr_'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data = 1*(data>0)
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [01:18<00:00,  3.18it/s] 


In [2]:
''' CTA and MRA for multiclass seg in the final version !!! '''

dataset_name = 'Dataset806_TopCoWSegCTAMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.endswith('.nii.gz'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data[data==15] = 13
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [05:33<00:00,  1.33s/it]


In [3]:
''' CTA and MRA for multiclass seg in the final version !!! '''

dataset_name = 'Dataset816_TopCoWSegMRACTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.endswith('.nii.gz'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data[data==15] = 13
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


  0%|          | 0/250 [00:00<?, ?it/s]

100%|██████████| 250/250 [01:49<00:00,  2.29it/s]


In [ ]:
''' Binary CTA with CTAMRA training '''

dataset_name = 'Dataset807_TopCoWSegBinCTAMRA' #'Dataset703_TopCoWDetCTAextendedMask' #'Dataset701_TopCoWDetCTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.endswith('.nii.gz'):
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data = 1*(data>0)
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [03:57<00:00,  1.05it/s]


In [4]:
''' Binary CTAMRA training (general) '''

dataset_name = 'Dataset811_TopCoWSegBinMRACTA' #'Dataset703_TopCoWDetCTAextendedMask' #'Dataset701_TopCoWDetCTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.endswith('.nii.gz'):
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data = 1*(data>0)
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [03:55<00:00,  1.06it/s]


In [ ]:
''' Multiclass MRA only '''

dataset_name = 'Dataset808_TopCoWSegMRA' #'Dataset703_TopCoWDetCTAextendedMask' #'Dataset701_TopCoWDetCTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.startswith('topcow_mr_'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data[data==15] = 13
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [01:07<00:00,  3.69it/s] 


In [3]:
''' Multiclass MRA only '''

dataset_name = 'Dataset818_TopCoWSegMRAablation' #'Dataset703_TopCoWDetCTAextendedMask' #'Dataset701_TopCoWDetCTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.startswith('topcow_mr_'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data[data==15] = 13
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [02:15<00:00,  1.84it/s]


In [8]:
''' Multiclass CTA only '''

dataset_name = 'Dataset815_TopCoWSegCTA' #'Dataset703_TopCoWDetCTAextendedMask' #'Dataset701_TopCoWDetCTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.startswith('topcow_ct_'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data[data==15] = 13
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [00:30<00:00,  8.24it/s]


##### Task 2 - CoW Detection

In [2]:

def get_bounding_box_3d(mask):
    """
    Given a 3D binary mask, returns the bounding box size and location in the format:
    (size_x, size_y, size_z), (min_x, min_y, min_z)
    """
    slices = find_objects(mask)
    if slices and slices[0] is not None:
        min_x, min_y, min_z = slices[0][0].start, slices[0][1].start, slices[0][2].start
        max_x, max_y, max_z = slices[0][0].stop, slices[0][1].stop, slices[0][2].stop
        return min_x, max_x, min_y, max_y, min_z, max_z
    else:
        return None, None

# Extend in the low part:
def extend_mask(seg_mask, roi_mask):
    seg_min_x, seg_max_x, seg_min_y, seg_max_y, seg_min_z, seg_max_z = get_bounding_box_3d(seg_mask)
    roi_min_x, roi_max_x, roi_min_y, roi_max_y, roi_min_z, roi_max_z = get_bounding_box_3d(roi_mask)

    if roi_min_x < seg_min_x:
        seg_mask[roi_min_x:seg_min_x, :, :] = np.repeat(seg_mask[seg_min_x:seg_min_x+1, :, :], seg_min_x - roi_min_x, axis=0)
    if roi_min_y < seg_min_y:
        seg_mask[:, roi_min_y:seg_min_y, :] = np.repeat(seg_mask[:, seg_min_y:seg_min_y+1, :], seg_min_y - roi_min_y, axis=1)
    if roi_min_z < seg_min_z:
        seg_mask[:, :, roi_min_z:seg_min_z] = np.repeat(seg_mask[:, :, seg_min_z:seg_min_z+1], seg_min_z - roi_min_z, axis=2)

    # Extend in the high part:
    if roi_max_x > seg_max_x:
        seg_mask[seg_max_x:roi_max_x, :, :] = np.repeat(seg_mask[seg_max_x-1:seg_max_x, :, :], roi_max_x - seg_max_x, axis=0)
    if roi_max_y > seg_max_y:
        seg_mask[:, seg_max_y:roi_max_y, :] = np.repeat(seg_mask[:, seg_max_y-1:seg_max_y, :], roi_max_y - seg_max_y, axis=1)
    if roi_max_z > seg_max_z:
        seg_mask[:, :, seg_max_z:roi_max_z] = np.repeat(seg_mask[:, :, seg_max_z-1:seg_max_z], roi_max_z - seg_max_z, axis=2)

    return seg_mask


# print(seg_min_x, seg_max_x, seg_min_y, seg_max_y, seg_min_z, seg_max_z)
# print(roi_min_x, roi_max_x, roi_min_y, roi_max_y, roi_min_z, roi_max_z)

In [3]:
''' MRA Box task with MRA only '''

dataset_name = 'Dataset704_TopCoWDetMRAextendedMask'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.startswith('topcow_mr_'):  #FIXME: 'topcow_ct_'            
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # shutil.copy(init_name, new_name)

        bbox_image = nib.load(os.path.join(data_folder, 'roi_masks', sub)).get_fdata().astype(np.uint8)
        nifti_image = nib.load(os.path.join(data_folder, 'cow_seg_labelsTr', sub))
        binary_image = (nifti_image.get_fdata() > 0).astype(int)
        cropped_mask = binary_image*bbox_image

        extended_mask = extend_mask(cropped_mask, bbox_image)
        # nib.save(nib.Nifti1Image(extended_mask.astype('int32'), affine=nifti_image.affine), os.path.join(data_path, subfold[1], sub))


  0%|          | 0/250 [00:00<?, ?it/s]

100%|██████████| 250/250 [07:24<00:00,  1.78s/it]


In [ ]:
''' CTA Box task with CTA + MRA '''

dataset_name = 'Dataset705_TopCoWDetCTAextendedMaskCTAMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


cta_imgs = '/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/Dataset703_TopCoWDetCTACropExtendedMulSegMask/imagesTr'
cta_labs = '/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/Dataset703_TopCoWDetCTACropExtendedMulSegMask/labelsTr'
mra_imgs = '/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/Dataset704_TopCoWDetMRAextendedMask/imagesTr'
mra_labs = '/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/Dataset704_TopCoWDetMRAextendedMask/labelsTr'

for file in tqdm(natsorted(os.listdir(cta_labs))):
    init_name = os.path.join(cta_imgs, file.split('.')[0]+'_0000.nii.gz')
    new_name = os.path.join(train_image_dir, file.split('.')[0]+'_0000.nii.gz')
    # # shutil.copy(init_name, new_name)

    init_name = os.path.join(cta_labs, file)
    new_name = os.path.join(train_label_dir, file)
    # # shutil.copy(init_name, new_name)


for file in tqdm(natsorted(os.listdir(mra_labs))):
    init_name = os.path.join(mra_imgs, file.split('.')[0]+'_0000.nii.gz')
    new_name = os.path.join(train_image_dir, file.split('.')[0]+'_0000.nii.gz')
    # # shutil.copy(init_name, new_name)

    init_name = os.path.join(mra_labs, file)
    new_name = os.path.join(train_label_dir, file)
    # # shutil.copy(init_name, new_name)


#### *Preparing dataset.json file*

In [7]:
dataset_name = 'Dataset705_TopCoWDetCTAextendedMaskCTAMRA' #'Dataset704_TopCoWDetMRAextendedMask'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Multiclass CoW roi Detection task using CTA but trained on CTA+MRA and bin label mask cropped from the multiclass GT and extended to have ROI bbox, trained for 5 folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "ctamra",
        # "1": "binmask",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "cow" : 1,
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


In [5]:
### CLEANED TOPCOW_CHALLENGE FOLDER AGAIN #3

dataset_name = 'Dataset806_TopCoWSegCTAMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Multiclass CoW segmentation task for CTA using CTA+MRA, trained for all folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "ctamra",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "BA" : 1,
        "R-PCA" : 2,
        "L-PCA" : 3,
        "R-ICA" : 4,
        "R-MCA" : 5,
        "L-ICA" : 6,
        "L-MCA" : 7,
        "R-Pcom" : 8,
        "L-Pcom" : 9,
        "Acom" : 10,
        "R-ACA" : 11,
        "L-ACA" : 12,
        "3rd-A2" : 13
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


In [4]:
### CLEANED TOPCOW_CHALLENGE FOLDER AGAIN #3

dataset_name = 'Dataset816_TopCoWSegMRACTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Multiclass CoW segmentation task using CTA+MRA, trained for all folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "ctamra",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "BA" : 1,
        "R-PCA" : 2,
        "L-PCA" : 3,
        "R-ICA" : 4,
        "R-MCA" : 5,
        "L-ICA" : 6,
        "L-MCA" : 7,
        "R-Pcom" : 8,
        "L-Pcom" : 9,
        "Acom" : 10,
        "R-ACA" : 11,
        "L-ACA" : 12,
        "3rd-A2" : 13
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


In [3]:
### CLEANED TOPCOW_CHALLENGE FOLDER AGAIN #3

dataset_name = 'Dataset807_TopCoWSegBinCTAMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Binary CoW segmentation task for CTA using CTA+MRA, trained for all folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "ctamra",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "cow" : 1        
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


In [5]:
### Binary CTAMRA training, w ith default configs

dataset_name = 'Dataset811_TopCoWSegBinMRACTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Binary CoW segmentation task using CTA+MRA (general), trained for all folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "ctamra",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "cow" : 1        
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


In [4]:
### CLEANED TOPCOW_CHALLENGE FOLDER AGAIN #3

dataset_name = 'Dataset808_TopCoWSegMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Multiclass CoW segmentation task for MRA, trained on 5 folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "mra",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "BA" : 1,
        "R-PCA" : 2,
        "L-PCA" : 3,
        "R-ICA" : 4,
        "R-MCA" : 5,
        "L-ICA" : 6,
        "L-MCA" : 7,
        "R-Pcom" : 8,
        "L-Pcom" : 9,
        "Acom" : 10,
        "R-ACA" : 11,
        "L-ACA" : 12,
        "3rd-A2" : 13
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


In [5]:
### CLEANED TOPCOW_CHALLENGE FOLDER AGAIN #3

dataset_name = 'Dataset818_TopCoWSegMRAablation'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Multiclass CoW segmentation task for MRA, trained on 5 folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "mra",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "BA" : 1,
        "R-PCA" : 2,
        "L-PCA" : 3,
        "R-ICA" : 4,
        "R-MCA" : 5,
        "L-ICA" : 6,
        "L-MCA" : 7,
        "R-Pcom" : 8,
        "L-Pcom" : 9,
        "Acom" : 10,
        "R-ACA" : 11,
        "L-ACA" : 12,
        "3rd-A2" : 13
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


In [9]:
### Multiclass segmentation using CTA only (from scratch)

dataset_name = 'Dataset815_TopCoWSegCTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Multiclass CoW segmentation task for CTA, trained on 5 folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "cta",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "BA" : 1,
        "R-PCA" : 2,
        "L-PCA" : 3,
        "R-ICA" : 4,
        "R-MCA" : 5,
        "L-ICA" : 6,
        "L-MCA" : 7,
        "R-Pcom" : 8,
        "L-Pcom" : 9,
        "Acom" : 10,
        "R-ACA" : 11,
        "L-ACA" : 12,
        "3rd-A2" : 13
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


In [3]:
### Binary CTA only

dataset_name = 'Dataset809_TopCoWSegBinCTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Binary CoW segmentation task for CTA using CTA only, trained for all folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "cta",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "cow" : 1        
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


In [7]:
### Binary MRA only

dataset_name = 'Dataset810_TopCoWSegBinMRAnew'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Binary CoW segmentation task for MRA using MRA only, trained for all folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "mra",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "cow" : 1        
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


#### Extra information

In [9]:
''' Information about Spacings and Sizes of all modalities '''

dataset_name = 'Dataset701_TopCoWDetCTA' 
# Renaming according to nnUNet: 
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'
subfold = ['imagesTr', 'cow_seg_labelsTr']

mri_spacing, seg_spacing = [], []
mri_size, seg_size = [], []
labels = []
fnames = []

# imageTr:
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.endswith('.nii.gz'):        
        mri = sitk.ReadImage(os.path.join(data_folder, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz"))
        mri_spacing.append(mri.GetSpacing())
        mri_size.append(mri.GetSize())
        
        seg = sitk.ReadImage(os.path.join(data_folder, subfold[1], sub))
        seg_spacing.append(seg.GetSpacing())
        seg_size.append(seg.GetSize())

        fnames.append(sub.split('.')[0])
        labels.append(sitk.GetArrayFromImage(seg).max())

df = pd.DataFrame({'filename':fnames, 'mri_spacing':mri_spacing, 'seg_spacing':seg_spacing, 'mri_size':mri_size, 'seg_size':seg_size, 'max_label':labels})
# df.to_csv('/home/hasna/miccai24_challenges/topcow_challenge/topcow24_dataset_spacing_size.tsv', sep='\t')

100%|██████████| 250/250 [03:22<00:00,  1.23it/s]


In [3]:
''' Information about Spacings and Sizes of all modalities '''

data_folder = '/home/hasna/datasets/crown_20_mr_01062023'

mri_spacing, seg_spacing = [], []
mri_size, seg_size = [], []
labels = []
fnames = []

# imageTr:
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'mul_labelsTr')))):
    if sub.startswith('crown_mr_whole'):        
        mri = sitk.ReadImage(os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz"))
        mri_spacing.append(mri.GetSpacing())
        mri_size.append(mri.GetSize())
        
        seg = sitk.ReadImage(os.path.join(data_folder, 'mul_labelsTr', sub))
        seg_spacing.append(seg.GetSpacing())
        seg_size.append(seg.GetSize())

        fnames.append(sub.split('.')[0])
        labels.append(sitk.GetArrayFromImage(seg).max())

df = pd.DataFrame({'filename':fnames, 'mri_spacing':mri_spacing, 'seg_spacing':seg_spacing, 'mri_size':mri_size, 'seg_size':seg_size, 'max_label':labels})
# df.to_csv('/home/hasna/miccai24_challenges/topcow_challenge/crown23_dataset_spacing_size.tsv', sep='\t')

100%|██████████| 40/40 [00:18<00:00,  2.22it/s]


### *Training nnUNet*

In [ ]:
'''
export nnUNet_raw='/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw'
export nnUNet_preprocessed='/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/preprocessed'
export nnUNet_results='/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/datasetnnUNet_trained_models'


>>> For task 1:
nnUNetv2_extract_fingerprint -d 802 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 802 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 802 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...


nnUNetv2_extract_fingerprint -d 806 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 806 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 806 -c 3d_fullres_ps -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...


nnUNetv2_extract_fingerprint -d 807 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 807 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 807 -c 3d_fullres_ps -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...


nnUNetv2_extract_fingerprint -d 808 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 808 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 808 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...


nnUNetv2_extract_fingerprint -d 809 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 809 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 809 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...

nnUNetv2_extract_fingerprint -d 810 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 810 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 810 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...

nnUNetv2_extract_fingerprint -d 811 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 811 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 811 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...



nnUNetv2_extract_fingerprint -d 815 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 815 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 815 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...

nnUNetv2_extract_fingerprint -d 816 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 816 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 816 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...



nnUNetv2_extract_fingerprint -d 818 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 818 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 818 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...



>>> For task 2:
nnUNetv2_extract_fingerprint -d 704 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 704 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 704 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...


nnUNetv2_extract_fingerprint -d 705 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 705 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 705 -c 3d_fullres_spacing_ps -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...

'''

"\nexport nnUNet_raw='/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/dataset/nnUNet_raw'\nexport nnUNet_preprocessed='/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/dataset/preprocessed'\nexport nnUNet_results='/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/datasetnnUNet_trained_models'\n\n\n>>> For task 1:\nnnUNetv2_extract_fingerprint -d 802 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'\nnnUNetv2_plan_experiment -d 802 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'\nnnUNetv2_preprocess -d 802 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...\n\n\nnnUNetv2_extract_fingerprint -d 806 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'\nnnUNetv2_plan_experiment -d 806 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'\nnnUNetv2_preprocess -d 806 -c

In [ ]:
'''
export nnUNet_raw='/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/dataset/nnUNet_raw'
export nnUNet_preprocessed='/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/dataset/preprocessed'
export nnUNet_results='/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/datasetnnUNet_trained_models'
'''

''' For task 2: -------------------------------------------------------------------------------------------
>>> Finetuning 704:  MAKE SURE THE SPLIT HAS NO DATA LEAKAGE
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 704 3d_fullres 0 -tr nnUNetTrainerSkeletonRecall -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset102_Binseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 704 3d_fullres 1 -tr nnUNetTrainerSkeletonRecall -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset102_Binseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_1/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 704 3d_fullres 2 -tr nnUNetTrainerSkeletonRecall -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset102_Binseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_2/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 704 3d_fullres 3 -tr nnUNetTrainerSkeletonRecall -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset102_Binseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_3/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 704 3d_fullres 4 -tr nnUNetTrainerSkeletonRecall -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset102_Binseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_4/checkpoint_best.pth

>>> Training 704:  3folds
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 704 3d_fullres 0 -tr nnUNetTrainerSkeletonRecall
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 704 3d_fullres 1 -tr nnUNetTrainerSkeletonRecall
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 704 3d_fullres 2 -tr nnUNetTrainerSkeletonRecall


>>> Finetuning 705:  MAKE SURE THE SPLIT HAS NO DATA LEAKAGE
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 705 3d_fullres_spacing_ps 0 -tr nnUNetTrainerSkeletonRecall -pretrained_weights /home/hasna/miccai24_challenges/epvs_challenge/nnunet_dir/dataset/preprocessed/Dataset702_TopCoWDetCTACroppedMulSegMask/pretrained_checkpoints/checkpoint_best_fold0.pth
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 705 3d_fullres_spacing_ps 1 -tr nnUNetTrainerSkeletonRecall -pretrained_weights /home/hasna/miccai24_challenges/epvs_challenge/nnunet_dir/dataset/preprocessed/Dataset702_TopCoWDetCTACroppedMulSegMask/pretrained_checkpoints/checkpoint_best_fold1.pth
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 705 3d_fullres_spacing_ps 2 -tr nnUNetTrainerSkeletonRecall -pretrained_weights /home/hasna/miccai24_challenges/epvs_challenge/nnunet_dir/dataset/preprocessed/Dataset702_TopCoWDetCTACroppedMulSegMask/pretrained_checkpoints/checkpoint_best_fold2.pth
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 705 3d_fullres_spacing_ps 3 -tr nnUNetTrainerSkeletonRecall -pretrained_weights /home/hasna/miccai24_challenges/epvs_challenge/nnunet_dir/dataset/preprocessed/Dataset702_TopCoWDetCTACroppedMulSegMask/pretrained_checkpoints/checkpoint_best_fold3.pth
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 705 3d_fullres_spacing_ps 4 -tr nnUNetTrainerSkeletonRecall -pretrained_weights /home/hasna/miccai24_challenges/epvs_challenge/nnunet_dir/dataset/preprocessed/Dataset702_TopCoWDetCTACroppedMulSegMask/pretrained_checkpoints/checkpoint_best_fold4.pth
'''

''' For task 1: -------------------------------------------------------------------------------------------
>>> [CTA with CTAMRA] !!RETRAINED without mirror!!  Training 806 from scratch, with BinaryDice loss: 
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 806 3d_fullres_ps 0 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 806 3d_fullres_ps 1 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 806 3d_fullres_ps 2 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 806 3d_fullres_ps 3 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 806 3d_fullres_ps 4 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring

CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 807 3d_fullres_ps all -tr nnUNetTrainerSkeletonRecall


>>> [MRA] !!RETRAINED without mirror!!  Training 808 from scratch, with BinaryDice loss: 
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 808 3d_fullres 0 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset101_Mulseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 808 3d_fullres 1 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset101_Mulseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_1/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 808 3d_fullres 2 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset101_Mulseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_2/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 808 3d_fullres 3 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset101_Mulseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_3/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 808 3d_fullres 4 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset101_Mulseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_4/checkpoint_best.pth



[BINARY RETRAINING]: ------------------------------------------------------------------------------
>>> Binary segmentation using CTA only
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 809 3d_fullres all -tr nnUNetTrainerSkeletonRecall --val_best
# TODO: A REFAIRE FOR 500 ONLY.. (it was correct.. just need 500)
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 809 3d_fullres 0 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 809 3d_fullres 1 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 809 3d_fullres 2 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 809 3d_fullres 3 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 809 3d_fullres 4 -tr nnUNetTrainerSkeletonRecall --val_best


>>> Binary segmentation using MRA only
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 810 3d_fullres all -tr nnUNetTrainerSkeletonRecall --val_best
# TODO: A REFAIRE
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 810 3d_fullres 0 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 810 3d_fullres 1 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 810 3d_fullres 2 -tr nnUNetTrainerSkeletonRecall --val_best
#REDO # CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 810 3d_fullres 3 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 810 3d_fullres 4 -tr nnUNetTrainerSkeletonRecall --val_best

>>> Binary segmentation using CTA + MRA
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 811 3d_fullres all -tr nnUNetTrainerSkeletonRecall --val_best
# TODO: A REFAIRE
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 811 3d_fullres 0 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 811 3d_fullres 1 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 811 3d_fullres 2 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 811 3d_fullres 3 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 811 3d_fullres 4 -tr nnUNetTrainerSkeletonRecall --val_best
## CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 811 3d_fullres 4 -tr nnUNetTrainerSkeletonRecall --val_best --c

[MULTICLASS RETRAINING]: ----------------------------------------------------------------------------------
>>> [CTA] !!RETRAINED without mirror!!  Training 815 from scratch, with BinaryDice loss: 
# TODO: A REFAIRE FOR SOME OF THEM TO HAVE 500 !!
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 815 3d_fullres 0 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
# CUDA_VISIBLE_DEVICES= nnUNetv2_train 815 3d_fullres 1 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 815 3d_fullres 2 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 815 3d_fullres 3 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
# CUDA_VISIBLE_DEVICES= nnUNetv2_train 815 3d_fullres 4 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best


>>> [CTAMRA] !!RETRAINED without mirror!!  Training 816 from scratch, with BinaryDice loss: 
# TODO: A REFAIRE
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 816 3d_fullres 0 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 816 3d_fullres 1 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 816 3d_fullres 2 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 816 3d_fullres 3 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 816 3d_fullres 4 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best



# FIXME: later
>>> [MRA] !!RETRAINED without mirror!!  Training 808 with default loss: 
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 0 -tr nnUNetTrainerNoMirroring --val_best
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 1 -tr nnUNetTrainerNoMirroring --val_best
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 2 -tr nnUNetTrainerNoMirroring --val_best
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 3 -tr nnUNetTrainerNoMirroring --val_best
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 4 -tr nnUNetTrainerNoMirroring --val_best

>>> [MRA] !!RETRAINED without mirror!!  Training 808 with default + SR losses: # TODO: Check the weights for the losses !!
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 808 3d_fullres 0 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 1 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 2 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 3 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 4 -tr nnUNetTrainerSkeletonRecall --val_best

>>> [MRA] !!RETRAINED without mirror!!  Training 808 with default + SR + BinDice losses: 
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 808 3d_fullres 0 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best --c
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 1 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 808 3d_fullres 2 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best --c
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 3 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES= nnUNetv2_train 808 3d_fullres 4 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best




>>> [MRA] !!RETRAINED without mirror!!  Training 818 with default + SR + BinDice losses: 
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 818 3d_fullres 0 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 818 3d_fullres 1 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 818 3d_fullres 2 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 818 3d_fullres 3 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 818 3d_fullres 4 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best

>>> [MRA] !!RETRAINED without mirror!!  Training 818 with default + SR losses: # TODO: Check the weights for the losses !!
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 818 3d_fullres 0 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 818 3d_fullres 1 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 818 3d_fullres 2 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 818 3d_fullres 3 -tr nnUNetTrainerSkeletonRecall --val_best
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 818 3d_fullres 4 -tr nnUNetTrainerSkeletonRecall --val_best

>>> [MRA] !!RETRAINED without mirror!!  Training 818 with default loss: 
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 818 3d_fullres 0 -tr nnUNetTrainerNoMirroring --val_best
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 818 3d_fullres 1 -tr nnUNetTrainerNoMirroring --val_best
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 818 3d_fullres 2 -tr nnUNetTrainerNoMirroring --val_best
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 818 3d_fullres 3 -tr nnUNetTrainerNoMirroring --val_best
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 818 3d_fullres 4 -tr nnUNetTrainerNoMirroring --val_best




## CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 809 3d_fullres  -tr nnUNetTrainerSkeletonRecall --val_best --c  # Done
## CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 810 3d_fullres  -tr nnUNetTrainerSkeletonRecall --val_best --c  # Done
## CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 811 3d_fullres  -tr nnUNetTrainerSkeletonRecall --val_best --c  # Done
## CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 815 3d_fullres  -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best --c  # Done
## CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 816 3d_fullres  -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best --c  # Done
## CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 818 3d_fullres 3 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring --val_best --c

'''

### Docker

In [ ]:
'''
cd /home/hasna/miccai24_challenges/TopCoW_Algo_Submission/task-2-box   #/task-1-seg

bash test_run.sh
bash save.sh

'''

### Metrics

In [ ]:
### CROWN 2023 Dataset (20 MRA)

import os
import shutil
from tqdm import tqdm
from natsort import natsorted

pred_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/predictions'
save_gt_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/ground-truth'
save_roi_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/roi-metadata'

os.makedirs(pred_folder, exist_ok=True)
os.makedirs(save_gt_folder, exist_ok=True)
os.makedirs(save_roi_folder, exist_ok=True)

seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge_final/evals/mra_crown23_skr_bindice_nomir_4folds/without_pp'
gt_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/CROWN23/mul_labelsTr'
roi_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/CROWN23/roi_size_loc'

for file in tqdm(natsorted(os.listdir(seg_folder))):
    if file.endswith('.nii.gz'):
        init_name = os.path.join(seg_folder, file)
        new_name = os.path.join(pred_folder, file.split('_0000')[0]+'.nii.gz')
        shutil.copy(init_name, new_name)

        init_name = os.path.join(gt_folder, file.split('_0000')[0]+'.nii.gz')
        new_name = os.path.join(save_gt_folder, file.split('_0000')[0]+'.nii.gz')
        # shutil.copy(init_name, new_name)

        init_name = os.path.join(roi_folder, file.replace("whole", "roi").split('_0000')[0]+'.txt')
        new_name = os.path.join(save_roi_folder, file.split('_0000')[0]+'.txt')
        # shutil.copy(init_name, new_name)


'''
REPLACE NUMBER IN HERE: 
/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py
'''

100%|██████████| 20/20 [00:00<00:00, 3633.48it/s]


'\nREPLACE NUMBER IN HERE: \n/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py\n'

In [ ]:
### MRA 2024 Dataset (125 MRA)

import os
import shutil
from tqdm import tqdm
from natsort import natsorted

pred_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/predictions'
save_gt_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/ground-truth'
save_roi_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/roi-metadata'

os.makedirs(pred_folder, exist_ok=True)
os.makedirs(save_gt_folder, exist_ok=True)
os.makedirs(save_roi_folder, exist_ok=True)

seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge_final/evals/mra_skr_bindice_nomir_5folds/with_pp_20'
gt_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/cow_seg_labelsTr'
roi_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/roi_loc_labelsTr'
nfolds = 5

for i in range(nfolds):
    for file in tqdm(natsorted(os.listdir(os.path.join(seg_folder, f'fold_{i}')))):
        if file.endswith('.nii.gz'):
            init_name = os.path.join(seg_folder, f'fold_{i}', file)
            new_name = os.path.join(pred_folder, file)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(gt_folder, file)
            new_name = os.path.join(save_gt_folder, file)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(roi_folder, file.split('.')[0]+'.txt')
            new_name = os.path.join(save_roi_folder, file.split('.')[0]+'.txt')
            # shutil.copy(init_name, new_name)


'''
REPLACE NUMBER IN HERE: 
/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py
'''

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:00<00:00, 743.23it/s]


'\nREPLACE NUMBER IN HERE: \n/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py\n'

In [ ]:
### MRA 2024 Dataset (125 MRA) BINARY

import os
import shutil
import nibabel as nib
from tqdm import tqdm
from natsort import natsorted

pred_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/predictions'
save_gt_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/ground-truth'
save_roi_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/roi-metadata'

os.makedirs(pred_folder, exist_ok=True)
os.makedirs(save_gt_folder, exist_ok=True)
os.makedirs(save_roi_folder, exist_ok=True)

# seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge/evals/cta_skr_bindice_nomir_5folds/bin'
seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge/evals/mra_skr_bindice_nomir_5folds_ctamra/without_pp'  # mra_skr_bindice_nomir_5folds_binary_ctamra/bin
gt_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/cow_seg_labelsTr'
roi_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/roi_loc_labelsTr'
nfolds = 5  #FIXME: 5 for MRA, and 1 for CTA

for i in range(nfolds):
    for file in tqdm(natsorted(os.listdir(os.path.join(seg_folder, f'fold_{i}')))):  #FIXME: For MRA
    # # for file in tqdm(natsorted(os.listdir(os.path.join(seg_folder)))):  #FIXME: For CTA
        if file.endswith('.nii.gz'):
            # # file = file.split('_0000')[0]+'.nii.gz'  #FIXME: For CTA
            init_name = os.path.join(seg_folder, f'fold_{i}', file)  #FIXME: For MRA
            # # init_name = os.path.join(seg_folder, file.split('.')[0]+'_0000.nii.gz')  #FIXME: For CTA
            new_name = os.path.join(pred_folder, file)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(gt_folder, file)
            # # img = nib.load(init_name)
            # # arr = 1*(img.get_fdata() > 0)
            new_name = os.path.join(save_gt_folder, file)
            # # nib.save(nib.Nifti1Image(arr, img.affine), new_name)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(roi_folder, file.split('.')[0]+'.txt')
            new_name = os.path.join(save_roi_folder, file.split('.')[0]+'.txt')
            # shutil.copy(init_name, new_name)


'''
REPLACE NUMBER IN HERE: 
/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py
'''

In [10]:
### MRA 2024 Dataset (125 MRA) BINARY from New Training

import os
import shutil
import nibabel as nib
from tqdm import tqdm
from natsort import natsorted

pred_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/predictions'
save_gt_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/ground-truth'
save_roi_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/roi-metadata'

os.makedirs(pred_folder, exist_ok=True)
os.makedirs(save_gt_folder, exist_ok=True)
os.makedirs(save_roi_folder, exist_ok=True)

seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/datasetnnUNet_trained_models/Dataset810_TopCoWSegBinMRAnew/nnUNetTrainerSkeletonRecall__nnUNetPlans__3d_fullres/fold_all/validation'
gt_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/cow_seg_labelsTr'
roi_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/roi_loc_labelsTr'
nfolds = 1

for file in tqdm(natsorted(os.listdir(os.path.join(seg_folder)))):
    if file.startswith('topcow_mr_'):
        init_name = os.path.join(seg_folder, file)
        new_name = os.path.join(pred_folder, file)
        # shutil.copy(init_name, new_name)

        init_name = os.path.join(gt_folder, file)
        img = nib.load(init_name)
        arr = 1*(img.get_fdata() > 0)
        new_name = os.path.join(save_gt_folder, file)
        # nib.save(nib.Nifti1Image(arr, img.affine), new_name)
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(roi_folder, file.split('.')[0]+'.txt')
        new_name = os.path.join(save_roi_folder, file.split('.')[0]+'.txt')
        # shutil.copy(init_name, new_name)


'''
REPLACE NUMBER IN HERE: 
/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py
'''

  0%|          | 0/126 [00:00<?, ?it/s]/tmp/ipykernel_3166620/1835575168.py:32: FutureWarning: Image data has type int64, which may cause incompatibilities with other tools. This will error in NiBabel 5.0. This warning can be silenced by passing the dtype argument to Nifti1Image().
  nib.save(nib.Nifti1Image(arr, img.affine), new_name)
100%|██████████| 126/126 [03:24<00:00,  1.62s/it]


'\nREPLACE NUMBER IN HERE: \n/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py\n'

In [ ]:
### MRA 2024 Dataset (125 MRA) MULTICLASS from New Training
### MRA 2024 Dataset (125 MRA) BINARY

import os
import shutil
import nibabel as nib
from tqdm import tqdm
from natsort import natsorted

pred_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/predictions'
save_gt_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/ground-truth'
save_roi_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/roi-metadata'

os.makedirs(pred_folder, exist_ok=True)
os.makedirs(save_gt_folder, exist_ok=True)
os.makedirs(save_roi_folder, exist_ok=True)

# seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/datasetnnUNet_trained_models/Dataset816_TopCoWSegMRACTA/nnUNetTrainerSkeletonRecallBinDiceNoMirroring__nnUNetPlans__3d_fullres'
# seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/datasetnnUNet_trained_models/Dataset818_TopCoWSegMRAablation/nnUNetTrainerSkeletonRecallBinDiceNoMirroring__nnUNetPlans__3d_fullres'
gt_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/cow_seg_labelsTr'
roi_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/roi_loc_labelsTr'
nfolds = 5

for i in range(nfolds):
    for file in tqdm(natsorted(os.listdir(os.path.join(seg_folder, f'fold_{i}/validation')))):
        if file.startswith('topcow_mr_'):
            init_name = os.path.join(seg_folder, f'fold_{i}/validation', file)
            new_name = os.path.join(pred_folder, file)
            # img = nib.load(init_name)
            # arr = img.get_fdata()
            # arr[arr == 13] = 15
            # nib.save(nib.Nifti1Image(arr, img.affine), new_name)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(gt_folder, file)
            # img = nib.load(init_name)
            # arr = 1*(img.get_fdata() > 0)
            new_name = os.path.join(save_gt_folder, file)
            # nib.save(nib.Nifti1Image(arr, img.affine), new_name)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(roi_folder, file.split('.')[0]+'.txt')
            new_name = os.path.join(save_roi_folder, file.split('.')[0]+'.txt')
            # shutil.copy(init_name, new_name)

'''
REPLACE NUMBER IN HERE: 
/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py
'''

100%|██████████| 26/26 [00:34<00:00,  1.32s/it]


'\nREPLACE NUMBER IN HERE: \n/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py\n'

In [ ]:
### Ablation + post-processing assessment
### MRA 2024 Dataset (125 MRA) MULTICLASS from New Training
### MRA 2024 Dataset (125 MRA) BINARY

import os
import shutil
import nibabel as nib
from tqdm import tqdm
from natsort import natsorted

pred_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/predictions'
save_gt_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/ground-truth'
save_roi_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/roi-metadata'

os.makedirs(pred_folder, exist_ok=True)
os.makedirs(save_gt_folder, exist_ok=True)
os.makedirs(save_roi_folder, exist_ok=True)

# seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/datasetnnUNet_trained_models/Dataset816_TopCoWSegMRACTA/nnUNetTrainerSkeletonRecallBinDiceNoMirroring__nnUNetPlans__3d_fullres'
# seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge/evals/MRA_D818-def-skel_D810/with_pp_20' #MRA_D818-all_D810/with_pp_20'  # MRA_D818-default_D810, MRA_D818-def-skel-bin_D810, MRA_D818-def-skel_D810
seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge/evals/CTA_D806-all-best_D809-all_1000ep/with_pp_20' #'/home/hasna/miccai24_challenges/topcow_challenge/evals/806_cta_skr_bindice_nomir_5folds_val-best/without_pp' #'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/datasetnnUNet_trained_models/Dataset806_TopCoWSegCTAMRA/nnUNetTrainerSkeletonRecallBinDiceNoMirroring__nnUNetPlans__3d_fullres_ps' #'/home/hasna/miccai24_challenges/topcow_challenge/evals/CTA_D806-all_D809-all_1000ep/with_pp_20' #'/home/hasna/miccai24_challenges/topcow_challenge/evals/CTA_D815-all_D809-all_1000ep/with_pp_20'
gt_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/cow_seg_labelsTr'
roi_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/roi_loc_labelsTr'
nfolds = 5  #5

for i in range(nfolds):
    for file in tqdm(natsorted(os.listdir(os.path.join(seg_folder, f'fold_{i}')))):  # FIXME: for 5 folds
    # for file in tqdm(natsorted(os.list/dir(os.path.join(seg_folder, 'validation')))):  # FIXME: for -all
        if file.startswith('topcow_ct_'):
            init_name = os.path.join(seg_folder, f'fold_{i}', file)  # FIXME: for 5 folds
            # init_name = os.path.join(seg_folder, f'validation', file)  # FIXME: for -all
            new_name = os.path.join(pred_folder, file)
            img = nib.load(init_name)
            arr = img.get_fdata()
            arr[arr == 13] = 15
            # nib.save(nib.Nifti1Image(arr, img.affine), new_name)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(gt_folder, file)
            # img = nib.load(init_name)
            # arr = 1*(img.get_fdata() > 0)
            new_name = os.path.join(save_gt_folder, file)
            # nib.save(nib.Nifti1Image(arr, img.affine), new_name)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(roi_folder, file.split('.')[0]+'.txt')
            new_name = os.path.join(save_roi_folder, file.split('.')[0]+'.txt')
            # shutil.copy(init_name, new_name)

'''
REPLACE NUMBER IN HERE: 
/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py
'''

100%|██████████| 25/25 [00:56<00:00,  2.28s/it]


'\nREPLACE NUMBER IN HERE: \n/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py\n'

In [2]:
### FINAL VERSION for Paper

import os
import shutil
import nibabel as nib
from tqdm import tqdm
from natsort import natsorted

pred_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/predictions'
save_gt_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/ground-truth'
save_roi_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/roi-metadata'

os.makedirs(pred_folder, exist_ok=True)
os.makedirs(save_gt_folder, exist_ok=True)
os.makedirs(save_roi_folder, exist_ok=True)

seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge/evals/final_CTA_bin809_mul806_skr-bindice_nomir_5folds_val-best/with_pp'   #with_pp
gt_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/cow_seg_labelsTr'
roi_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/roi_loc_labelsTr'
nfolds = 5  # final_CTA_bin809_mul806_skr-bindice_nomir_5folds_val-best_with_pp

for i in range(nfolds):
    for file in tqdm(natsorted(os.listdir(os.path.join(seg_folder, f'fold_{i}')))):  # FIXME: for 5 folds
        if file.startswith('topcow_ct_'):
            init_name = os.path.join(seg_folder, f'fold_{i}', file)  # FIXME: for 5 folds
            new_name = os.path.join(pred_folder, file)
            img = nib.load(init_name)
            arr = img.get_fdata()
            arr[arr == 13] = 15
            nib.save(nib.Nifti1Image(arr, img.affine), new_name)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(gt_folder, file)
            # img = nib.load(init_name)
            # arr = 1*(img.get_fdata() > 0)
            new_name = os.path.join(save_gt_folder, file)
            # nib.save(nib.Nifti1Image(arr, img.affine), new_name)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(roi_folder, file.split('.')[0]+'.txt')
            new_name = os.path.join(save_roi_folder, file.split('.')[0]+'.txt')
            # shutil.copy(init_name, new_name)

'''
REPLACE NUMBER IN HERE: 
/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py
'''

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:15<00:00,  1.66it/s]


'\nREPLACE NUMBER IN HERE: \n/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py\n'

In [ ]:
### MRA 2024 Dataset (2 folds)

import os
import shutil
from tqdm import tqdm
from natsort import natsorted

pred_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/predictions'
save_gt_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/ground-truth'
save_roi_folder = '/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/roi-metadata'

os.makedirs(pred_folder, exist_ok=True)
os.makedirs(save_gt_folder, exist_ok=True)
os.makedirs(save_roi_folder, exist_ok=True)

# mra_skr_bindice_withmir_fold-1_with_pp
# mra_skr_bindice_withmir_fold-1_without_pp
seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge_final/evals/mra_skr_bindice_withmir_2folds/without_pp'

# mra_skr_bindice_nomir_fold-1_without_pp
# mra_skr_bindice_nomir_fold-1_with_pp
# seg_folder = '/home/hasna/miccai24_challenges/topcow_challenge_final/evals/mra_skr_bindice_nomir_5folds/without_pp'
# gt_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/cow_seg_labelsTr'
# roi_folder = '/home/hasna/datasets/TopCoW2024_Data_Release/roi_loc_labelsTr'
# nfolds = 1

for i in [1]: #range(nfolds):
    for file in tqdm(natsorted(os.listdir(os.path.join(seg_folder, f'fold_{i}')))):
        if file.endswith('.nii.gz'):
            init_name = os.path.join(seg_folder, f'fold_{i}', file)
            new_name = os.path.join(pred_folder, file)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(gt_folder, file)
            new_name = os.path.join(save_gt_folder, file)
            # shutil.copy(init_name, new_name)

            init_name = os.path.join(roi_folder, file.split('.')[0]+'.txt')
            new_name = os.path.join(save_roi_folder, file.split('.')[0]+'.txt')
            # shutil.copy(init_name, new_name)


'''
REPLACE NUMBER IN HERE: 
/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py
'''

100%|██████████| 25/25 [00:00<00:00, 3813.42it/s]


'\nREPLACE NUMBER IN HERE: \n/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/topcow24_eval/configs.py\n'

In [4]:
import os
import json
import numpy as np
import pandas as pd

with open('/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/output/metrics_mra_skr_bindice_nomir_5folds_with_pp.json', 'r') as file:
    metrics_with_pp = json.load(file)
with open('/home/hasna/miccai24_challenges/TopCoW_Eval_Metrics/output/metrics_mra_skr_bindice_nomir_5folds_without_pp.json', 'r') as file:
    metrics_without_pp = json.load(file)
with open('/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/dataset/preprocessed/Dataset808_TopCoWSegMRA/splits_final.json', 'r') as file:
    splits = json.load(file)

In [43]:
import ast
metrics_with_pp['case'].keys()
metrics_with_pp['case']['Dice_ClsAvgDice']
metrics_with_pp['case']['HD95_ClsAvgHD95']
metrics_with_pp['case']['clDice']


mets = ['Dice_ClsAvgDice', 'clDice']  #'HD95_ClsAvgHD95
df = pd.DataFrame(columns=['metric', 'index', 'val_with_pp', 'val_without_pp'])
for met in mets:
    for idx, val in metrics_with_pp['case'][met].items():
        if val < metrics_without_pp['case'][met][idx] - 0.015:
            new_row = pd.DataFrame({
                'metric': [met],
                'index': [int(idx)+1],
                'val_with_pp': [val],
                'val_without_pp': [metrics_without_pp['case'][met][idx]]
            })
            df = pd.concat([df, new_row], ignore_index=True)
df[df['metric'] == 'Dice_ClsAvgDice']


,metric,index,val_with_pp,val_without_pp
0,Dice_ClsAvgDice,7,0.801426,0.836272
1,Dice_ClsAvgDice,41,0.698666,0.763709
2,Dice_ClsAvgDice,93,0.899800,0.915267
3,Dice_ClsAvgDice,100,0.892367,0.909599
4,Dice_ClsAvgDice,110,0.880562,0.942338
5,Dice_ClsAvgDice,119,0.847899,0.871679


In [6]:
import ast
metrics_with_pp['case'].keys()
metrics_with_pp['case']['Dice_ClsAvgDice']
metrics_with_pp['case']['HD95_ClsAvgHD95']
metrics_with_pp['case']['clDice']


mets = ['Dice_ClsAvgDice', 'HD95_ClsAvgHD95', 'clDice']
df = pd.DataFrame(columns=['metric', 'index', 'val_with_pp', 'val_without_pp'])
for met in mets:
    for idx, val in metrics_with_pp['case'][met].items():
        if val < metrics_without_pp['case'][met][idx]:
            new_row = pd.DataFrame({
                'metric': [met],
                'index': [int(idx)+1],
                'val_with_pp': [val],
                'val_without_pp': [metrics_without_pp['case'][met][idx]]
            })
            df = pd.concat([df, new_row], ignore_index=True)
df[df['metric'] == 'Dice_ClsAvgDice']


,metric,index,val_with_pp,val_without_pp
0,Dice_ClsAvgDice,7,0.801426,0.836272
1,Dice_ClsAvgDice,8,0.922731,0.924010
2,Dice_ClsAvgDice,9,0.794300,0.795986
3,Dice_ClsAvgDice,13,0.807366,0.809266
4,Dice_ClsAvgDice,15,0.689932,0.694921
5,Dice_ClsAvgDice,17,0.848003,0.848094
6,Dice_ClsAvgDice,20,0.837560,0.839046
7,Dice_ClsAvgDice,25,0.892822,0.898452
8,Dice_ClsAvgDice,32,0.873819,0.878243
9,Dice_ClsAvgDice,34,0.797894,0.800841


In [ ]:
# l = (df[df['metric'] == 'Dice_ClsAvgDice']['index'])
# r = [[int(splits[j]['val'][i].split('_')[-1]) for i in range(len(splits[0]['val']))] for j in range(5)]

# target = [int(x) for x in list(l)]
# result = {}

# for num in target:
#     # Find which sublist contains the number
#     indices = [i for i, sublist in enumerate(r) if num in sublist]
#     result[num] = indices

# result

### Liu's Topology refinement

In [ ]:
'''

git clone https://github.com/smilell/Universal-Topology-Refinement.git

>>> For testing:
python /home/hasna/miccai24_challenges/Universal-Topology-Refinement/main.py --phase test_post --trained_model_post ./model_1/poly_1/ --out ./output_1/poly_1/ --TBout./output_1/poly_1/ --test_softmax ./data/cremi_2d/test/softmax/

'''

### Cleaning

In [ ]:
import os
from tqdm import tqdm
from natsort import natsorted

main_dir = '/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/preprocessed/Dataset811_TopCoWSegBinMRACTA/nnUNetPlans_3d_fullres'
for file in tqdm(natsorted(os.listdir(main_dir))):
    if file.endswith('.npy'):
        # os.remove(os.path.join(main_dir, file))

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:01<00:00, 824.93it/s]
